# Text 3: Latent Dirichlet allocation
**Internet Analytics - Lab 4**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
import json
from collections import defaultdict

## Exercise 4.8: Topics extraction

In [ ]:

!hdfs dfs -put courses.txt
data = sc.textFile("courses.txt").map(json.loads)
data = data.map(lambda x: x['listDescription'])
termCounts = data.flatMap(lambda x:x).map(lambda x : (x,1)).reduceByKey(lambda a,b:a+b).map(lambda x:(x[1],x[0])).sortByKey(False)
voc = termCounts.map(lambda x:x[1]).zipWithIndex().collectAsMap()

def document_vector(document):
    id = document[1]
    counts = defaultdict(int)
    for token in document[0]:
        if token in voc:
            token_id = voc[token]
            counts[token_id] += 1
    counts = sorted(counts.items())
    keys = [x[0] for x in counts]
    values = [x[1] for x in counts]
    return (id, Vectors.sparse(len(voc), keys, values))

documents = data.zipWithIndex().map(document_vector).map(list)
inv_voc = {value: key for (key, value) in voc.items()}

lda_model = LDA.train(documents)
result =lda_model.describeTopics()

In [ ]:
for i in range(10):
    print('topic',(i+1),'=================')
    for j in range(10):
        print(inv_voc[result[i][0][j]])

## Exercise 4.9: Dirichlet hyperparameters

## Exercise 4.10: EPFL's taught subjects

## Exercise 4.11: Wikipedia structure